In [119]:
import pandas as pd
import ast


df = pd.read_csv('movies_dataset_transformado(sin desanidar).csv')

# Elimina las columnas
columas_a_borrar = ["video","imdb_id","adult","original_title","poster_path","homepage"] 
df.drop(columas_a_borrar, inplace=True, axis=1)

# Elimina las filas en las que al columna tenga valores nulos
df.dropna(subset=['release_date'], inplace=True) 

# Pasa a CERO los valores nulos de la columna
df['budget'].fillna(0, inplace=True)
df['revenue'].fillna(0, inplace=True)

# Crea la columna requerida
df['return'] = df['revenue'] / df['budget']  

# Paso a formato Datetime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Paso a formato AAAA-mm-dd
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d')

# Creo la columna con el año de la fecha
df['release_year'] = df['release_date'].dt.year 


# Convertir las columnas de string a diccionarios/listas
columnas_anidadas = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages']
for columna in columnas_anidadas:
    df[columna] = df[columna].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else {})

# Desanidar las columnas
df_belongs_to_collection = pd.json_normalize(df['belongs_to_collection'])
df = pd.concat([df.drop('belongs_to_collection', axis=1), df_belongs_to_collection], axis=1)

df_genres = pd.json_normalize(df['genres'].explode()).groupby(level=0).agg(list)
df = pd.concat([df, df_genres], axis=1)
df.drop('genres', axis=1, inplace=True)


# Renombrar nuevas columnas
df.columns = ['id_collection', 'name_collection', "poster_path_collection", 'backdrop_path_collection', 'budget', 'id',
       'original_language', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'return', 'release_year', 'id_genres', 'name_genres']


# Expandir la columna production_countries
df_production_countries = pd.json_normalize(df['production_countries'].explode()).groupby(level=0).agg(list)
df = pd.concat([df.drop('production_countries', axis=1), df_production_countries], axis=1)


# Expandir la columna production_companies
df_production_companies = pd.json_normalize(df['production_companies'].explode()).groupby(level=0).agg(list)
df = pd.concat([df.drop('production_companies', axis=1), df_production_companies], axis=1)


# Renombrar nuevas columnas
df.columns = [['id_collection', 'name_collection', 'poster_path_collection',
       'backdrop_path_collection', 'budget', 'id', 'original_language',
       'overview', 'popularity', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'return', 'release_year', 'id_genres', 'name_genres',
       'iso_3166_1_country', 'country']]

# Eliminar columnas innecesarias
df = df.drop(columns=['spoken_languages'])

# Guardar el DataFrame en un archivo CSV
df.to_csv('movies_dataset_transformado.csv', index=False)


# Cargar el nuevo df 
df_credits = pd.read_csv('credits.csv')


# Convertir las columnas de string a diccionarios/listas
columnas_anidadas = ['cast', 'crew']

for columna in columnas_anidadas:
    df_credits[columna] = df_credits[columna].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else {})


# Desanidar la columna 'cast'
df_cast = pd.json_normalize(df_credits['cast'].explode())


# Desanidar la columna 'crew'
df_crew = pd.json_normalize(df_credits['crew'].explode())


# Ordenar los DF por las columnas
df_cast = df_cast.sort_values(by='cast_id', ascending=True)
df_crew = df_crew.sort_values(by='credit_id', ascending=True)


# Eliminar columnas innecesarias
dfprueba_cast = df_cast.drop(columns=['order', 'profile_path',"gender"])
df_crew = df_crew.drop(columns=['profile_path',"gender"])


# Ordenar columnas
dfprueba_cast = dfprueba_cast[['cast_id', 'id', 'name', 'character', 'credit_id']]
df_crew = df_crew[['credit_id', 'id', 'name', 'department', 'job']]


# Guardar los dfs en un archivo CSV
df_crew.to_csv('crew_dataset.csv', index=False)
df_cast.to_csv('cast_dataset.csv', index=False)